In [ ]:
import wandb
from wandb import Api
import pandas as pd
import os
import shutil

# === CONFIGURAZIONE ===
sweeps_mapping = {
    "7ovklbta" : "YOLOv11s", # Sweep Bayesian
    "4q1q0ld1" : "YOLOv11s", # Sweep with the initial run
    "7e7521i8": "YOLOv11n",  # Sweep Bayesian
    "so0hcj67": "YOLOv11n",  # Sweep with the initial run 

}

ENTITY = "pothole-detector"
PROJECT = "pothole-detector-NatureSR"

# === LOGIN ===
wandb.login()

# === SETUP ===
api = Api()

In [ ]:
# === ESTRAZIONE DEI DATI ===
all_runs_data = []

summary_useful = [
    'lr/pg0', 
    'lr/pg1', 
    'lr/pg2', 
    'metrics/mAP50(B)', 
    'metrics/mAP50-95(B)', 
    'metrics/precision(B)', 
    'metrics/recall(B)', 
    'model/GFLOPs', 
    'model/parameters', 
    'model/speed_PyTorch(ms)', 
    'train/box_loss', 
    'train/cls_loss', 
    'train/dfl_loss',
    'val/box_loss',
    'val/cls_loss', 
    'val/dfl_loss',
]

# === CICLO SU TUTTI GLI SWEEP ===
OUTPUT_DIR = {sweep_id: f"results/{name}" for sweep_id, name in sweeps_mapping.items()}
CONFIG_DIR = "configs"
os.makedirs(CONFIG_DIR, exist_ok=True)

for sweep_id in sweeps_mapping:
    out_dir = OUTPUT_DIR[sweep_id]
    os.makedirs(out_dir, exist_ok=True)
    print(f"\n📦 Sweep {sweep_id} → output in '{out_dir}'")
    
    try:
        sweep = api.sweep(f"{ENTITY}/{PROJECT}/{sweep_id}")
        runs = sweep.runs
        print(f"🔍 Trovati {len(runs)} run")
        
        for run in runs:
            run_id = run.id
            run_name = run.name or run_id
            found_files = []
            
            print(f"🔄 Analizzo run: {run_name}")
            
            # === Scarica anche il config.yaml del run ===
            try:
                files = run.files()
                if "config.yaml" in [f.name for f in files]:
                    config_file = run.file("config.yaml")
                    config_path = os.path.join(CONFIG_DIR, f"{run_name}_{run_id}_config.yaml")
                    config_file.download(CONFIG_DIR, replace=True)
                    # rinomina se serve
                    downloaded_path = os.path.join(CONFIG_DIR, "config.yaml")
                    if os.path.exists(downloaded_path):
                        os.rename(downloaded_path, config_path)
                    print(f"   ⚙️ Config salvato: {config_path}")
                else:
                    print("   ⚠️ Nessun config.yaml trovato per questo run")
            except Exception as e:
                print(f"   ❌ Errore scaricando config.yaml: {e}")
                exit(0)
            
            # === Elenco artifacts come prima ===
            artifacts = run.logged_artifacts()
            print(f"   📁 Trovati {len(artifacts)} artifacts")
            
            for artifact in artifacts:
                if not artifact.name.split(':')[0].endswith('_log'):
                    continue
                    
                print(f"   🔍 Controllo artifact: {artifact.name}")
                
                try:
                    artifact_dir = artifact.download()
                    print(f"   ⬇️ Scaricato in: {artifact_dir}")
                    
                    for root, dirs, files in os.walk(artifact_dir):
                        for file in files:
                            if file == "results.csv":
                                results_path = os.path.join(root, file)
                                artifact_name_clean = artifact.name.replace(":", "_").replace("/", "_")
                                output_filename = f"{run_name}_{run_id}_{artifact_name_clean}_results.csv"
                                output_path = os.path.join(out_dir, output_filename)
                                
                                shutil.copy(results_path, output_path)
                                print(f"   ✅ Salvato: {output_path}")
                                found_files.append(output_path)
                                
                except Exception as e:
                    print(f"   ❌ Errore scaricando artifact {artifact.name}: {e}")
            
            if not found_files:
                print("   ⚠️ Nessun 'results.csv' trovato in nessun artifact.")
            else:
                print(f"   🎉 Trovati {len(found_files)} file results.csv")
                
    except Exception as e:
        print(f"❌ Errore nello sweep {sweep_id}: {e}")

print("\n🏁 Download completato!")